# Assignment 5
## Group Members:
### Nils Dunlop, e-mail: gusdunlni@student.gu.se
### Francisco Alejandro Erazo Piza, e-mail: guserafr@student.gu.se
### Chukwudumebi Ubogu, e-mail: gusuboch@student.gu.se
***

### Task 1: A small linear regression example in PyTorch
***

#### Task 1 Discussion
***
Our model demonstrated substantial improvement over the training period, with the MSE reducing from an initial value of 0.7999 in epoch 1 to approximately 0.0094 by epoch 10. This progression indicates successful parameter optimization, aligning with the expected outcome of a correctly implemented linear regression model in PyTorch.

### Task 2: Implementing tensor arithmetics for forward computations
***

We decided to centralize tasks 2-4 in the next cell because we faced several challenges when trying to replicate and adjust the code across multiple cells for each task.  Initially, we copied the code from previous steps to incorporate these new requirements. But this strategy made it harder to keep track of changes and increased the likelihood of mistakes. 

### Task 4: Implementing the backward computations
***

### Task 5: Optimizers to update the model parameters
***

### Task 6: Classifying raisins
***